In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))
import pandas as pd
import numpy as np
import time

t = time.time()

In [2]:
df_train = pd.read_csv('data/df_train.csv')
df_test = pd.read_csv('data/df_test.csv')
assert (df_test.columns == df_train.columns).all()
print(df_train.columns)
print(len(df_train.columns))

Index(['Unnamed: 0', 'day', 'hashed_partner_id', 'hashed_campaign_id',
       'contextid', 'display_env', 'target_env', 'rtbtypeid',
       'rtbadvisibility', 'rtb_detectedlanguage', 'urlhash2', 'urlhash3',
       'urlhash4', 'user_country', 'hashed_affiliateid', 'hashed_app_id',
       'googleviewability', 'googlepagevertical', 'campaignscenario',
       'campaignvertical', 'campaignctrlast24h', 'is_interstitial',
       'dayssincelastvisitdouble', 'ltf_lastpartnerclicktimestamp',
       'ltf_nbglobalclick_4w', 'ltf_nbglobaldisplay_4w',
       'ltf_nbglobaldisplaysincelastpartnerproductview',
       'ltf_nbpartnerdisplayssincelastclick', 'ltf_nbpartnerclick_4w',
       'ltf_nbpartnerdisplay_4w', 'ltf_nbpartnersales_4w',
       'ltf_nbpartnerdisplay_90d', 'ltf_nbpartnerclick_90d',
       'ltf_nbpartnersales_90d', 'nbdayssincelastclick', 'nbdisplay_1hour',
       'nbdisplaypartnerapprox_1d_sum_xdevice',
       'nbdisplayaffiliateapprox_1d_sum_xdevice',
       'nbdisplayglobalapprox_1d_s

In [3]:
def recoding(df):
    # Suppression de la variable "Unnamed: 0"
    df.drop('Unnamed: 0', inplace=True, axis=1)

    # Recodage de certaines modalités afin de réordonner par ordre croissant d'engagement
    df["contextid"] = df["contextid"].replace(0,1)
    df["contextid"] = df["contextid"].replace(10,3)

    # Remplacement des valeurs manquantes par des 0 dans les variables commençant par ltf
    df['ltf_lastpartnerclicktimestamp'] = df['ltf_lastpartnerclicktimestamp'].fillna(0)
    df['ltf_nbglobalclick_4w'] = df['ltf_nbglobalclick_4w'].fillna(0)
    df['ltf_nbglobaldisplay_4w'] = df['ltf_nbglobaldisplay_4w'].fillna(0)
    df['ltf_nbglobaldisplaysincelastpartnerproductview'] = df['ltf_nbglobaldisplaysincelastpartnerproductview'].fillna(0)
    df['ltf_nbpartnerdisplayssincelastclick'] = df['ltf_nbpartnerdisplayssincelastclick'].fillna(0)
    df['ltf_nbpartnerclick_4w'] = df['ltf_nbpartnerclick_4w'].fillna(0)
    df['ltf_nbpartnerdisplay_4w'] = df['ltf_nbpartnerdisplay_4w'].fillna(0)
    df['ltf_nbpartnersales_4w'] = df['ltf_nbpartnersales_4w'].fillna(0)
    df['ltf_nbpartnerdisplay_90d'] = df['ltf_nbpartnerdisplay_90d'].fillna(0)
    df['ltf_nbpartnerclick_90d'] = df['ltf_nbpartnerclick_90d'].fillna(0)
    df['ltf_nbpartnersales_90d'] = df['ltf_nbpartnersales_90d'].fillna(0)

    # Idem ici
    df['nbdisplay_1hour'] = df['nbdisplay_1hour'].fillna(0)
    df['nbdisplaypartnerapprox_1d_sum_xdevice'] = df['nbdisplaypartnerapprox_1d_sum_xdevice'].fillna(0)
    df['nbdisplayaffiliateapprox_1d_sum_xdevice'] = df['nbdisplayaffiliateapprox_1d_sum_xdevice'].fillna(0)
    df['nbdisplayglobalapprox_1d_sum_xdevice'] = df['nbdisplayglobalapprox_1d_sum_xdevice'].fillna(0)	
    df['campaignctrlast24h'] = df['campaignctrlast24h'].fillna(0)

    # On n'utilisera que la surface de la publicité
    df['display_size'] = df['display_width'] * df['display_height']
    df.drop('display_width', inplace=True, axis=1)
    df.drop('display_height', inplace=True, axis=1)

recoding(df_train)
recoding(df_test)

In [4]:
# Variables avec valeurs manquantes dans la base d'entraînement
pd.DataFrame({"Valeurs manquantes":df_train[df_train.columns[df_train.isnull().sum() != 0]].isnull().sum(), "Proportion manquante":df_train[df_train.columns[df_train.isnull().sum() != 0]].isnull().sum()/len(df_train)})

,Valeurs manquantes,Proportion manquante
rtbtypeid,9066,0.004887
rtbadvisibility,9066,0.004887
rtb_detectedlanguage,9066,0.004887
urlhash2,155695,0.083918
urlhash3,155695,0.083918
urlhash4,155695,0.083918
user_country,366,0.000197
hashed_app_id,736418,0.396924
googleviewability,1278860,0.689296
googlepagevertical,9066,0.004887


In [5]:
# On retrouve les mêmes proportions de valeurs manquantes dans la base de test
pd.DataFrame({"Valeurs manquantes":df_test[df_test.columns[df_test.isnull().sum() != 0]].isnull().sum(), "Proportion manquante":df_test[df_test.columns[df_test.isnull().sum() != 0]].isnull().sum()/len(df_test)})

,Valeurs manquantes,Proportion manquante
rtbtypeid,1603,0.005726
rtbadvisibility,1603,0.005726
rtb_detectedlanguage,1603,0.005726
urlhash2,24429,0.087269
urlhash3,24429,0.087269
urlhash4,24429,0.087269
user_country,81,0.000289
hashed_app_id,112934,0.403441
googleviewability,192892,0.689080
googlepagevertical,1603,0.005726


In [6]:
def addTimestamp(df):
    df['timestamp'] = pd.to_datetime(df.display_timestamp, unit='s')
    df['hour'] = df.timestamp.dt.hour
    df['weekday'] = df.timestamp.dt.day_name()
    df.drop('display_timestamp', inplace=True, axis=1)

addTimestamp(df_train)
addTimestamp(df_test)

In [7]:
# détéction des éventuels appareils cliquant un nombre aberrant de fois (robots)
def nbClicsDevice(df):
    b = df[['is_display_clicked', 'hashed_xd_id']].groupby(['hashed_xd_id']).sum()
    b.sort_values('is_display_clicked', ascending=False, inplace=True)
    display(b)

In [8]:
# un appareil a un comportement anormal dans la base d'entraînement
nbClicsDevice(df_train)

,is_display_clicked
hashed_xd_id,
b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8',14472
"b""L\xf5H\x97\x10\xfd\x7f~\x1a9\xa3?\xac\xfdNf\xf6\\\x8f13\xc1\xcc\xf8\xbf\xc9q\x87'\xbc]H""",10
b'\x97\x9d)\xb3*\x8e\xf6\xeer\xf7\x166a\x12\xcc\xa5\xf4\x90\xc9\xd9\x197\xd1\xdf\x17f\x10%\xfe\x99\xe4\xb1',10
"b""\xbaI`\x80\xf3\x0b\xa4\xbc\x84'\x1f\xdeA\xecv\xbb\xa6:W\xf3\xe8\xcb\xf6~L\x91\xfeS\x90q\xcb~""",9
b'>\x90\xdd\x05s\xf2\x8a\x1dv\xc8Kdy\xbc\xd9\xe7a+\xf6\x04G2\xed\x1eT\xe6\xaeS\x81L$\x9c',9
...,...
b'\x05\xa3\x85\xf7TeZ\x98\xcc&\x15\xdc\xbaY\x85\xe8\xf3\xce\xd2I\xc8R:;-\x1f\xb9\xb9\x9e\xef\x89\x95',0
"b'\x05\xa3\x99\xafx4\x96,G\xca\x03I\x97C|\xe7~\xad\x1d\x80M\x96[\xf0\xbd\x886\x96\x1c\xb8\x9c\xa9'",0
b'\x05\xa3\xaa?\xa0W\xf5\xc2\xf4\xcev~X\xe6\xfc\xb3%\xc8?;T\xff\xc1\x02\xf5\\k\xcb2N[\n',0


In [9]:
# ce même appareil apparaît aussi dans la base de test
nbClicsDevice(df_test)

,is_display_clicked
hashed_xd_id,
b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8',2269
b'`B\xb3\xb6\xab\xb9V\x1f#<\x91\x0cq\xd8\xbb\xa8\x08\xb6\xc5\x9ar\xbd\xdce\x88\x078yl:\xee1',4
b'\x02\xdbB\xbd[\xfc\xb1\x85\xa0J\xd9\x94\xf3\xf5k\\\x0fT/\x9cW\x0b\xef\xef\x9ak\t\x84\n\x88\xba\xf7',4
b'\xb0\xba?\xd3Dn1W\xe2\xc7\xb2\x947\xba\xaaA\x15Q\xcc|\xaf\xe0\xb5g\xa9\xc0\x07)\xd5\xf6/\xf4',4
b'\x87\xf4\x99\xa8\x96\xb8\x8d\xaay\xd1\xa0\x82\x0c\xc5\x94\x8cF)*y\x18T\xf8\xc8\xde\xbfwhc\xe3\xe1\xbf',4
...,...
b'\x05@c\x1a^\x18w\xc5\x9b\xe7\xa6p\x10?\xcd?\xc4\x94\x04\xe6\x1e\xe2T\xd7u\xbfv\xa0\xfb\xd7\x1eI',0
b'\x05D\xf2^\xd5\xc4\xd2\xf0-t\xcd\xc0@\xe5z\xac\x1ay\xbe\xff\xea\x86\x83\xa6\xa8\x82d:V\xe6\\D',0
b'\x05E`\x13|i%8G\xf8\x84m\xc1wn\xb5\xf2L\x8f\xb4\x11<\x8c\xb8B\x99\xbc[\xe8\xb2A\xe0',0


In [10]:
# Suppression de l'appareil cliquant un nombre aberrant de fois dans les deux bases
df_train.drop(df_train.index[df_train['hashed_xd_id'] == r"b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8'"], inplace = True)
df_test.drop(df_test.index[df_test['hashed_xd_id'] == r"b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8'"], inplace = True)

In [11]:
display(df_train.head())
display(df_test.head())
assert (df_test.columns == df_train.columns).all()
print(len(df_train.columns))

,day,hashed_partner_id,hashed_campaign_id,contextid,display_env,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,...,hashed_uid,hashed_xd_id,valueperclick,device_type,is_display_clicked,zonecostineuro,display_size,timestamp,hour,weekday
0,2020-10-19,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,"b""f|g\xd5T\xc8\xe2\x90\xc6\xc8\xfeL\x0c\x15\xe...",6,web,1,74.0,0.0,127.0,1.697895e+08,...,b'\x99w[\xb8\xe1\x9e\xbaz\x01\x06\x9c\xdc\x03\...,b'\xdb\xfc\xdfBY`$\x97\xf9\xaaGn\x98\xe0{RQ\x9...,0.529218,Android,0,0.366238,0,2020-10-19 23:36:30,23,Monday
1,2020-10-19,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,b'7\xa0%5\x98\x02\x00\xe5\xa2\xfb\xd6\xa9^\xb7...,1,web,1,74.0,0.0,127.0,3.589877e+08,...,b'>\x1a\x00W\xb0\x81\x06\x00\x90\xc8\x8aL\xb4\...,b'\xe9\xcd\x80\xb5\x96\xcf8\x8eo(p-\x0f\xf4\xf...,0.269752,Desktop,0,0.025551,0,2020-10-19 23:37:20,23,Monday
2,2020-10-19,"b'\xa1l\n\xb2`\xe3\x0b""\xcd\x06\xfa\xdf\x9aj0\...","b'\xbe\xa0?\xc2</,\xa4&w\x86\xe9\x150Y\xd4\xdf...",6,web,1,4.0,2.0,10.0,-1.859395e+09,...,b'o\xa5~{\xf5\xd0C\xac_\xa9\x10\x95\x04L\x8a\x...,b'o\xa5~{\xf5\xd0C\xac_\xa9\x10\x95\x04L\x8a\x...,0.095993,Android,0,0.289000,96000,2020-10-19 23:39:40,23,Monday
3,2020-10-19,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,b'7\xa0%5\x98\x02\x00\xe5\xa2\xfb\xd6\xa9^\xb7...,6,web,1,73.0,1.0,127.0,-8.462803e+08,...,b'\xf2\x93e\xae\xadBh\x9f<<\x9e\xf0\x91q\xba\x...,"b'\x83u\xce\x10n\x9f\x10hP\x0cI\xdb \xed\xa8,E...",0.269752,iPhone,0,0.243945,4,2020-10-19 23:40:40,23,Monday
4,2020-10-19,b'\x0f\xbc\x909\x14[dI\xa7v]\xcc\x00\xd3\xbd\x...,b'\xa0\t^\xa5Z\x06[-\x86\x13\xd8\x08\x98\x8de\...,8,app_ios,2,74.0,0.0,127.0,NaN,...,b'\xe6\xa9\xb2:\xc5\x04\xfd\x89/Lg\xc1\x0e\xa4...,b'\xe6\xa9\xb2:\xc5\x04\xfd\x89/Lg\xc1\x0e\xa4...,0.136416,iPhone,0,0.647304,0,2020-10-19 23:26:10,23,Monday


,day,hashed_partner_id,hashed_campaign_id,contextid,display_env,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,...,hashed_uid,hashed_xd_id,valueperclick,device_type,is_display_clicked,zonecostineuro,display_size,timestamp,hour,weekday
1,2020-10-20,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,b'G\x85\xe1u\x9a\x85\x158\xbdb\x91\xde2\x16\x0...,6,web,1,73.0,1.0,127.0,-2.684551e+08,...,"b""\xf0x]\x9a\xe7/\xd7=\xcd\x82&\x9b\xd1\xc4Qu:...","b""\xf0x]\x9a\xe7/\xd7=\xcd\x82&\x9b\xd1\xc4Qu:...",0.585255,Android,0,0.050001,75000,2020-10-20 23:33:40,23,Tuesday
3,2020-10-20,b'F\xb7$\xb2\xf8]O{\xd4\x85B:\x04\xbc\xf2\xcf\...,b'z+\xb4py\xb9\xd8\x1eG\xbf\x19\x00\x0e\xa8\xf...,9,web,1,4.0,2.0,24.0,-5.447171e+08,...,b'\x91\xdaT\xa4\x050RX>7m\xd3S*\xb1P\xf0\xbb\x...,b':\xaaXO=\xe67\xbaL\xed\xdb\xfa\x9c\xdb\xe4Yp...,0.187720,Android,0,1.880000,153600,2020-10-20 23:28:50,23,Tuesday
4,2020-10-20,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,b'Q\xba\x90%\xf8\xb0V\xdcM\xd4\r)1\x9d\x0f)w\x...,1,app_android,1,5.0,0.0,12.0,NaN,...,b'\x14x\x95\xf5Y\x85\xa7\xc32<n\xc5\xf4oF~\xfc...,"b'\xca\xb2F\x08\xba\x93\x1c\xbb\xb5,\xefL\xcf\...",0.050532,Android,0,0.016971,16000,2020-10-20 23:29:30,23,Tuesday
5,2020-10-20,b'\x91h\xe8G\x86\x14)#\r\xa31\xe2:\xa7\x988b\x...,b'L\xfdJ\xc6\xaf\x96\x1c@\xda2\x97b\xf4jLw\xfa...,9,app_ios,2,97.0,0.0,9.0,NaN,...,b'\x80V\xb4\xf9\xa1\xc4\x0bfh\x1b\x18\xde\x90<...,"b""\x9do\xda\x164\xa4\xfc'\x10\x8a\x88\xb8?M\xb...",0.171740,iPhone,0,0.432753,16000,2020-10-20 23:29:00,23,Tuesday
7,2020-10-20,b'=\xa6\xeef\x99\xda\x1e\xb5-5\x8a\xa5\x9b\x8e...,b'\xd9]\x0b\xd5\xa9\xc56\x06~F/a\xcd\xd6Z\x02\...,6,app_ios,2,4.0,1.0,12.0,1.862823e+09,...,b'\x95\x80\xcc\xca\x0c-j\x02;\x8a\xde\x9d\xbf\...,b'>\xbe\xc0\x0f\x9f\x03\xd0\xacc\x11tO\xd1t\x9...,0.112012,iPhone,0,0.031000,0,2020-10-20 23:34:30,23,Tuesday


48


In [12]:
df_train.to_csv('data/df_train_prepro.csv', index=False)

In [13]:
df_test.to_csv('data/df_test_prepro.csv', index=False)

In [14]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

Temps d'exécution total : 00:01:39
